## Automated Modelling Schema

This modelling schema includes full parametrization of the transformations, the objective is to achieve encapsulated gri-search viable pipeline  
to improve optimization processes and efficiency. Pipeline model allows us to export parameters of the entire pipeline instead of just model weights.

Includes:
- Imputation strategies (ETL)
- New columns processing (ETL)

In [1]:
import os
import sys

sys.dont_write_bytecode = True

import numpy as np
import pandas as pd

from etl import *

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

from dotenv import load_dotenv
load_dotenv()

False

loading raw data

In [2]:
DATA_DIR = '../Development/gulas16/Data/'
DATA_FILE = 'spotify_tracks_kaggle_weekly.csv'
ARTIST_FILE = 'spotify_tracks_artist_details.csv'

In [3]:
data_tracks = pd.read_csv(DATA_DIR + DATA_FILE)
data_artist = pd.read_csv(DATA_DIR + ARTIST_FILE)

Merge

In [4]:
data = pd.merge(data_tracks, data_artist, on='track_id', how='left')

Data splitting -> working with predefined $X_{train}$

In [5]:
RANDOM_STATE = 21
TEST_SIZE = 0.1

In [6]:
X = data.drop('popularity', axis=1)
y = data['popularity']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

Dropping columns defined in EDA

In [8]:
drop_columns = ['track_id', 'artwork_url', 'track_url', 'track_name']

In [9]:
X_train = X_train.drop(drop_columns, axis=1, errors='ignore')
X_test = X_test.drop(drop_columns, axis=1, errors='ignore')

transformation pipeline

In [10]:
target = 'popularity'

onehot_col = ['language']
circle_of_fifths_col = ['key']
artist_name_col = ['artist_name']
album_name_col = ['album_name']
genre_col = ['artist_genres']
follower_col = ['artist_followers']
artist_popularity_col = ['artist_popularities']


numeric_columns = list(X_train.columns[X_train.dtypes != object].difference(['key', 'mode']))

nan_columns = ['acousticness', 'danceability', 'energy', 'liveness', 'speechiness', 'tempo', 'valence']

In [11]:
numeric_pipeline = Pipeline(steps=[
    ('imputation', SimpleImputer()),
    ('polynomials', PolynomialFeatures()),
    ('scaling', StandardScaler())
])

artist_name_pipeline = Pipeline(steps=[
    ('encoding', FrequencyEncoder()),
    ('imputation', SimpleImputer()),
    ('scaling', StandardScaler())
])

album_name_pipeline = Pipeline(steps=[
    ('encoding', AlbumNameEncoder()),
    ('scaling', StandardScaler())
])

circle_of_fifths_pipeline = Pipeline(steps=[
    ('encoding', CircleOfFifthsEncoding()),
    ('imputation', SimpleImputer())
])

genre_pipeline = Pipeline(steps=[
    ('encoding', GenreEncoder()),
    ('scaling', StandardScaler())
])

followers_pipeline = Pipeline(steps=[
    ('encoding', FollowerCountEncoder()),
    ('scaling', StandardScaler())
])

artist_popularity_pipeline = Pipeline(steps=[
    ('encoding', ArtistPopularityEncoder()),
    ('scaling', StandardScaler())
])


transformations = ColumnTransformer(transformers=[
    
    ('onehot_encoding', OneHotEncoder(sparse_output=False), onehot_col),
    ('trigonometric_encoding', circle_of_fifths_pipeline, circle_of_fifths_col),
    ('artist_encoding', artist_name_pipeline, artist_name_col),
    ('album_encoding', album_name_pipeline, album_name_col),
    ('follower_encoding', followers_pipeline, follower_col),
    ('genres_encoding', genre_pipeline, genre_col),
    ('artist_popularity_encoding', artist_popularity_pipeline, artist_popularity_col),
    ('numeric_processing', numeric_pipeline, numeric_columns)

], remainder='drop').set_output(transform='pandas')


preprocessing = Pipeline(steps=[
    
    ('null_values', ConvertNull(columns=nan_columns)),
    ('transformation', transformations)

]).set_output(transform='pandas')

In [24]:
preprocessing

Pipeline(steps=[('null_values',
                 ConvertNull(columns=['acousticness', 'danceability', 'energy',
                                      'liveness', 'speechiness', 'tempo',
                                      'valence'])),
                ('transformation',
                 ColumnTransformer(transformers=[('onehot_encoding',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['language']),
                                                 ('trigonometric_encoding',
                                                  Pipeline(steps=[('encoding',
                                                                   CircleOfFifthsEncoding()),
                                                                  ('imputation',
                                                                   S...
                                                  ['artist_popularities']),
                                                 ('numeric_processing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('polynomials',
                                                                   PolynomialFeatures()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  ['acousticness',
                                                   'artist_count',
                                                   'avg_artist_popularity',
                                                   'danceability',
                                                   'duration_ms', 'energy',
                                                   'instrumentalness',
                                                   'liveness', 'loudness',
                                                   'speechiness', 'tempo',
                                                   'time_signature', 'valence',
                                                   'year'])]))])

### Model configuration

- select model
- provide appropriate hyperparameter selection
- autoamted optimization
- export best fit

In [79]:
model = None

In [13]:
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing),
    ('model', model)
])

NameError: name 'model' is not defined

optimization - hyperparameter tuning

In [81]:
from sklearn.model_selection import GridSearchCV

In [82]:
param_grid = {}
k_fold = 5

In [51]:
results = GridSearchCV(model_pipeline, param_grid=param_grid, cv=k_fold)